Learning how to build a chatbox. Following Tech with Time (https://www.youtube.com/watch?v=wypVcNIH6D4&list=PLzMcBGfZo4-ndH9FoC4YWHGXG5RZekt-Q)

In [2]:
import nltk
#nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

import warnings
warnings.filterwarnings('ignore')

The intent file is formated as a JSON file. Here is the format:
{"intents": [
        {"tag": "greating",
        "patterns": ["Hi"],
        "response" ["Hello"],
        "context_set": ""
        }
        
The basic structor is as follows:
    - person types a message
    - the chatbot, through deeplearning, tags the message
    - the chatbot then returns with one of the responses

In [4]:
# reading our json file
with open ('intents.json') as file:
    data = json.load(file)
    
#print(data)  # just to make sure we correctly loaded/read our file
#print(data['intents'])

## Preprocess Data

In [5]:
# save each patter/grouping into a variable 
words = []
labels = []
docs_x = [] # for each patter, we need to know what intent is
docs_y = [] 

for intent in data['intents']:
    for pattern in intent['patterns']:
        # stemming will break each word in our pattern and break it down into the root word
        # ex. "Is anyone there?": the root is "there" ignoreing the other words and ?
        # we need to tokkenize (getting all the words)
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])
        
    if intent['tag'] not in labels:
            labels.append(intent['tag'])
            

# stem all words and remove duplicates 
words = [stemmer.stem(w.lower()) for w in words if w != '?']
words = sorted(list(set(words))) # set removes duplicates, list converts set back to list, sorted just sorts the words

labels = sorted(labels)


To run data in a neurl network, you need to convert the string data into numbers. To do this, we'll convert our strings into "bag of words" (one-hot encoding)

In [18]:
training = []  # a list of 0 and 1. ex. [0,0,0,1,2,0,4,0] meaning there is 0 mention of 'a', 2 mention of "hi", etc
output = []  # a list of 0 and 1. ex. [0,0,0,1] meaning there is a mention of "goodby"

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    
    wrds = [stemmer.stem(w) for w in doc]
    
    # one-hot encoding
    for w in words:
        if w in wrds: 
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1  # look through the "label" list. See where the tag is in the list and place a 1

    training.append(bag)
    output.append(output_row)  # this is where i made the mistake. I coded: bag.append(output_row), should be ouput.app...
    
    
training = np.array(training)
output = np.array(output)

with open('data.pickle', 'wb') as f:
    pickle.dump((words, labels, training, output), f)

In [25]:
# verify if training and output are correctly one-hot encoded

print(training[4])
print(output[0])

print(len(training))
print(len(output))

print(len(training[1]))
print(len(output[5]))

[0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0]
26
26
46
6


In [27]:
tf.reset_default_graph()

# model layers
net = tflearn.input_data(shape=[None, len(training[0])]) # the shape of data
net = tflearn.fully_connected(net, 8) # hidden layer
net = tflearn.fully_connected(net, 8) # hidden layer
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net) # the network

In [28]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 3999  | total loss: 0.68710 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.68710 - acc: 0.9191 -- iter: 24/26
Training Step: 4000  | total loss: 0.62412 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.62412 - acc: 0.9272 -- iter: 26/26
--
